In [30]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

import pandas as pd
import openai
import os 
pd.options.display.max_colwidth = None



In [4]:

openai_api_key = os.environ.get('OPENAI_API_KEY')
print(len(openai_api_key))

51


In [5]:
from langchain.document_loaders.csv_loader import CSVLoader

In [7]:
loader = CSVLoader(file_path="ImmigrationCanada_Sponsorship_QA_may7.csv")

data = loader.load()

In [31]:
print((data[0].page_content))

question: I'm the PA, sponsored by my Canadian husband, an outland application. I'm a US citizen.

I got my PPR at the end of April and within a few days I had sent them everything they needed, EXCEPT an envelope in which to send me my CoPR...

I'm wondering two things: 1) Is it possible they'll have mercy on me and just grab a plain envelope to send my stuff back in? I sent them a return label for UPS, and I'd think they can just send the documents in any old envelope... and 2) How long will it take them to notify me if they will require me to send the things again?

I'm grinding my teeth over here trying to figure out how to be proactive in this situation. Should I submit a case-specific inquiry? Should I try to call somewhere? **Will my whole application be denied if it takes longer than 30 days (the required PPR turnaround time) to hear back from them?**

Last thing: I'm wondering about tracking. I login to the mycic dashboard to see my application status. I haven't been able to lo

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [19]:
persist_directory = 'docs/chroma/'


In [36]:
!rm -rf ./docs/chroma

In [21]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings,persist_directory=persist_directory)

In [22]:
question = "can i go out of the country while waiting for PR"

In [23]:
docs = vectordb.similarity_search(question,k=3)

In [26]:
print(docs[0].page_content)

question: Just filed PR application for my wife (got married abroad last month). I’m a PR living in Canada but she’s back home. Can I also apply for a tourist visa for her so she can come visit me while we wait for her PR application to go through?
answer: Yes. 

Officially the two applications do not have an effect on each other.

That said, it can be harder to convince an IRCC agent of your intention to return to your home country if your spouse is in Canada and you're in the process of wrapping up your life to move. For that reason, it can be difficult to obtain a visitor visa for spouses who are currently being sponsored. Not impossible, of course, but many people do have them refused.
question_length: 238
answer_length: 441


In [28]:
vectordb.persist()

In [32]:
print(vectordb._collection.count())

288
